In [1]:
# Import the Earth Engine Python Package
import ee
import numpy as np
import pandas as pd

# Initialize the Earth Engine object, using the authentication credentials.
ee.Initialize()




In [2]:
import sys
print(sys.executable)

?ee.batch.Export.table.toDrive

/anaconda3/bin/python


### Create imperviousness change function

In [2]:
def imperv_change(urban_region):
  # Look at change of imperviousness from 2001 to 2011
  
  nlcd_2001 = ee.Image( 'USGS/NLCD/NLCD2001')
  nlcd_2011 = ee.Image( 'USGS/NLCD/NLCD2011')
  
  # Get imperviousness layer
  nlcd_2001_imperv = nlcd_2001.select('impervious').clip( urban_region) 
  nlcd_2011_imperv = nlcd_2011.select('impervious').clip( urban_region)
  
  # Can we quantify how much Chicago grew in these 10 years?
  imperv_difference = nlcd_2011_imperv.subtract( nlcd_2001_imperv) 
  
  # Lose zero values - they don't interest us right now
  imperv_difference_masked = imperv_difference.updateMask( imperv_difference) 
  
  # Find the fraction of pixels in this region that show an increase in impermeability
    
  # ee.Image.pixelarea multiplies the value of each pixel by its area
  # So, setting each pixel value to 1 and multiplying by area gives
  # a map of pixel areas of all valid pixels
  urban_pixelarea = imperv_difference.gt(-999).multiply( ee.Image.pixelArea()) 
  
  increased_imperv_pixelarea = imperv_difference.gt( 0).multiply( ee.Image.pixelArea()) 
  
  # Reduceregions applies a reducer to all pixels in a region
  # In this case, we are getting the sum of all pixel areas
  # within each image, and extracting it as an ee.Numeric object
  increased_imperv_area = ee.Number(
      ee.List(
        increased_imperv_pixelarea
        .reduceRegions( urban_region, ee.Reducer.sum())
        .aggregate_array( 'sum')
      ).get(0)
    ) 
  
  urban_area =   ee.Number( ee.List(
        urban_pixelarea
        .reduceRegions( urban_region, ee.Reducer.sum())
        .aggregate_array( 'sum')
      ).get(0)
    ) 
  
  return increased_imperv_area.divide( urban_area)



### Run the function for a test region

In [3]:

# Define an urban region to test the function

urban_region = ee.Geometry.Polygon(
[-88.66173511290759,42.18796899676144,
  -88.58483081603259,41.63200862139874,
  -88.39806323790759,41.37282825888678,
  -87.52465015197009,41.37695033374023,
  -87.52465015197009,41.69766697753461,
  -87.50267749572009,41.738669446161744,
  -87.64549976134509,42.037194621449935,
  -87.80480151915759,42.27338406127144,
  -87.79381519103259,42.492491901277525,
  -88.44200855040759,42.492491901277525,
  -88.66173511290759,42.18796899676144]
)


In [4]:
imperv_change(urban_region).getInfo()

0.05903345928770684

### Find coordinate files and read into Python

In [5]:
import os
coord_directory = "data_out/csa_wkt"
files           = os.listdir( coord_directory)
files           = np.array( files)[ [f.endswith(".txt") for f in files] ]
paths           = [coord_directory + "/" + f for f in files]
paths


['data_out/csa_wkt/Charlotte-Concord, NC-SC.txt',
 'data_out/csa_wkt/Seattle-Tacoma, WA.txt',
 'data_out/csa_wkt/Philadelphia-Reading-Camden, PA-NJ-DE-MD.txt',
 'data_out/csa_wkt/Boston-Worcester-Providence, MA-RI-NH-CT.txt',
 'data_out/csa_wkt/New York-Newark, NY-NJ-CT-PA.txt',
 'data_out/csa_wkt/Chicago-Naperville, IL-IN-WI.txt',
 'data_out/csa_wkt/Detroit-Warren-Ann Arbor, MI.txt',
 'data_out/csa_wkt/San Jose-San Francisco-Oakland, CA.txt',
 'data_out/csa_wkt/Minneapolis-St. Paul, MN-WI.txt',
 'data_out/csa_wkt/Miami-Fort Lauderdale-Port St. Lucie, FL.txt',
 'data_out/csa_wkt/St. Louis-St. Charles-Farmington, MO-IL.txt',
 'data_out/csa_wkt/Washington-Baltimore-Arlington, DC-MD-VA-WV-PA.txt',
 'data_out/csa_wkt/Orlando-Deltona-Daytona Beach, FL.txt',
 'data_out/csa_wkt/Los Angeles-Long Beach, CA.txt',
 'data_out/csa_wkt/Pittsburgh-New Castle-Weirton, PA-OH-WV.txt',
 'data_out/csa_wkt/Portland-Vancouver-Salem, OR-WA.txt',
 'data_out/csa_wkt/Houston-The Woodlands, TX.txt',
 'data_out/c

### Loop over input files, creating geometries and running imperviousness function

In [6]:
change   = np.array([])
csa_name = np.array([])

# Loop over input files. Create earth engine geometry, run imperv_change function over
# this geometry, export imperviousness change and CSA name

for i in range(0,len(paths)):
    
    print( "Running file " + str(i) + " (", paths[i], ")")
    
    f = open( paths[i], 'r')
    urban_geometry_raw = f.read()
    urban_geometry     = [ float(n) for n in urban_geometry_raw.split()]
    urban_geometry_ee  = ee.Geometry.Polygon(urban_geometry)

    change = np.append( change, imperv_change( urban_geometry_ee ).getInfo() )
    csa_name = np.append( csa_name, files[i].split(', ')[0] )
    

Running file 0 ( data_out/csa_wkt/Charlotte-Concord, NC-SC.txt )
Running file 1 ( data_out/csa_wkt/Seattle-Tacoma, WA.txt )
Running file 2 ( data_out/csa_wkt/Philadelphia-Reading-Camden, PA-NJ-DE-MD.txt )
Running file 3 ( data_out/csa_wkt/Boston-Worcester-Providence, MA-RI-NH-CT.txt )
Running file 4 ( data_out/csa_wkt/New York-Newark, NY-NJ-CT-PA.txt )
Running file 5 ( data_out/csa_wkt/Chicago-Naperville, IL-IN-WI.txt )
Running file 6 ( data_out/csa_wkt/Detroit-Warren-Ann Arbor, MI.txt )
Running file 7 ( data_out/csa_wkt/San Jose-San Francisco-Oakland, CA.txt )
Running file 8 ( data_out/csa_wkt/Minneapolis-St. Paul, MN-WI.txt )
Running file 9 ( data_out/csa_wkt/Miami-Fort Lauderdale-Port St. Lucie, FL.txt )
Running file 10 ( data_out/csa_wkt/St. Louis-St. Charles-Farmington, MO-IL.txt )
Running file 11 ( data_out/csa_wkt/Washington-Baltimore-Arlington, DC-MD-VA-WV-PA.txt )
Running file 12 ( data_out/csa_wkt/Orlando-Deltona-Daytona Beach, FL.txt )
Running file 13 ( data_out/csa_wkt/Los 

In [7]:
change

array([0.02259457, 0.00658724, 0.02069067, 0.02161341, 0.02014194,
       0.02753243, 0.01414824, 0.00889902, 0.01651396, 0.01499798,
       0.01156153, 0.01513019, 0.02403795, 0.00872231, 0.01144897,
       0.0036172 , 0.02840517, 0.01946746])

In [8]:
csa_name


array(['Charlotte-Concord', 'Seattle-Tacoma',
       'Philadelphia-Reading-Camden', 'Boston-Worcester-Providence',
       'New York-Newark', 'Chicago-Naperville',
       'Detroit-Warren-Ann Arbor', 'San Jose-San Francisco-Oakland',
       'Minneapolis-St. Paul', 'Miami-Fort Lauderdale-Port St. Lucie',
       'St. Louis-St. Charles-Farmington',
       'Washington-Baltimore-Arlington', 'Orlando-Deltona-Daytona Beach',
       'Los Angeles-Long Beach', 'Pittsburgh-New Castle-Weirton',
       'Portland-Vancouver-Salem', 'Houston-The Woodlands',
       'Dallas-Fort Worth'], dtype='<U36')

In [12]:
output_file = pd.DataFrame( {"csa_name": csa_name,
                             "change":   change})
output_file


,csa_name,change
0,Charlotte-Concord,0.022595
1,Seattle-Tacoma,0.006587
2,Philadelphia-Reading-Camden,0.020691
3,Boston-Worcester-Providence,0.021613
4,New York-Newark,0.020142
5,Chicago-Naperville,0.027532
6,Detroit-Warren-Ann Arbor,0.014148
7,San Jose-San Francisco-Oakland,0.008899
8,Minneapolis-St. Paul,0.016514
9,Miami-Fort Lauderdale-Port St. Lucie,0.014998


In [16]:
output_file.to_csv( "data_out/csa_imperv_change.csv", index=False)